In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter
import os

from itertools import product
from colorama import Fore

from datetime import datetime, timedelta, time

from sklearn.linear_model import LinearRegression
import statsmodels as sm
import statsmodels.formula.api as smf

from statsmodels.tools.tools import maybe_unwrap_results
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.stats.outliers_influence import variance_inflation_factor

from typing import Type

import statsmodels

/home/h01/ldawkins/.conda/envs/climada_env_2022/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
"""
    Code to produce diagnostic plots for linear regression. copied from 
    https://www.statsmodels.org/dev/examples/notebooks/generated/linear_regression_diagnostics_plots.html  
"""


class Linear_Reg_Diagnostic():
    """
    Diagnostic plots to identify potential problems in a linear regression fit.
    Mainly,
        a. non-linearity of data
        b. Correlation of error terms
        c. non-constant variance
        d. outliers
        e. high-leverage points
        f. collinearity

    Author:
        Prajwal Kafle (p33ajkafle@gmail.com, where 3 = r)
        Does not come with any sort of warranty.
        Please test the code one your end before using.
    """

    def __init__(self,
                 results: Type[statsmodels.regression.linear_model.RegressionResultsWrapper]) -> None:
        """
        For a linear regression model, generates following diagnostic plots:

        a. residual
        b. qq
        c. scale location and
        d. leverage

        and a table

        e. vif

        Args:
            results (Type[statsmodels.regression.linear_model.RegressionResultsWrapper]):
                must be instance of statsmodels.regression.linear_model object

        Raises:
            TypeError: if instance does not belong to above object

        Example:
        >>> import numpy as np
        >>> import pandas as pd
        >>> import statsmodels.formula.api as smf
        >>> x = np.linspace(-np.pi, np.pi, 100)
        >>> y = 3*x + 8 + np.random.normal(0,1, 100)
        >>> df = pd.DataFrame({'x':x, 'y':y})
        >>> res = smf.ols(formula= "y ~ x", data=df).fit()
        >>> cls = Linear_Reg_Diagnostic(res)
        >>> cls(plot_context="seaborn-paper")

        In case you do not need all plots you can also independently make an individual plot/table
        in following ways

        >>> cls = Linear_Reg_Diagnostic(res)
        >>> cls.residual_plot()
        >>> cls.qq_plot()
        >>> cls.scale_location_plot()
        >>> cls.leverage_plot()
        >>> cls.vif_table()
        """

        if isinstance(results, statsmodels.regression.linear_model.RegressionResultsWrapper) is False:
            raise TypeError("result must be instance of statsmodels.regression.linear_model.RegressionResultsWrapper object")

        self.results = maybe_unwrap_results(results)

        self.y_true = self.results.model.endog
        self.y_predict = self.results.fittedvalues
        self.xvar = self.results.model.exog
        self.xvar_names = self.results.model.exog_names

        self.residual = np.array(self.results.resid)
        influence = self.results.get_influence()
        self.residual_norm = influence.resid_studentized_internal
        self.leverage = influence.hat_matrix_diag
        self.cooks_distance = influence.cooks_distance[0]
        self.nparams = len(self.results.params)

    def __call__(self, plot_context='seaborn-paper'):
        # print(plt.style.available)
        with plt.style.context(plot_context):
            fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
            self.residual_plot(ax=ax[0,0])
            self.qq_plot(ax=ax[0,1])
            self.scale_location_plot(ax=ax[1,0])
            self.leverage_plot(ax=ax[1,1])
            plt.show()

        self.vif_table()
        return fig, ax


    def residual_plot(self, ax=None):
        """
        Residual vs Fitted Plot

        Graphical tool to identify non-linearity.
        (Roughly) Horizontal red line is an indicator that the residual has a linear pattern
        """
        if ax is None:
            fig, ax = plt.subplots()

        sns.residplot(
            x=self.y_predict,
            y=self.residual,
            lowess=True,
            scatter_kws={'alpha': 0.5},
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        residual_abs = np.abs(self.residual)
        abs_resid = np.flip(np.sort(residual_abs))
        abs_resid_top_3 = abs_resid[:3]
        for i, _ in enumerate(abs_resid_top_3):
            ax.annotate(
                i,
                xy=(self.y_predict[i], self.residual[i]),
                color='C3')

        ax.set_title('Residuals vs Fitted', fontweight="bold")
        ax.set_xlabel('Fitted values')
        ax.set_ylabel('Residuals')
        return ax

    def qq_plot(self, ax=None):
        """
        Standarized Residual vs Theoretical Quantile plot

        Used to visually check if residuals are normally distributed.
        Points spread along the diagonal line will suggest so.
        """
        if ax is None:
            fig, ax = plt.subplots()

        QQ = ProbPlot(self.residual_norm)
        QQ.qqplot(line='45', alpha=0.5, lw=1, ax=ax)

        # annotations
        abs_norm_resid = np.flip(np.argsort(np.abs(self.residual_norm)), 0)
        abs_norm_resid_top_3 = abs_norm_resid[:3]
        for r, i in enumerate(abs_norm_resid_top_3):
            ax.annotate(
                i,
                xy=(np.flip(QQ.theoretical_quantiles, 0)[r], self.residual_norm[i]),
                ha='right', color='C3')

        ax.set_title('Normal Q-Q', fontweight="bold")
        ax.set_xlabel('Theoretical Quantiles')
        ax.set_ylabel('Standardized Residuals')
        return ax

    def scale_location_plot(self, ax=None):
        """
        Sqrt(Standarized Residual) vs Fitted values plot

        Used to check homoscedasticity of the residuals.
        Horizontal line will suggest so.
        """
        if ax is None:
            fig, ax = plt.subplots()

        residual_norm_abs_sqrt = np.sqrt(np.abs(self.residual_norm))

        ax.scatter(self.y_predict, residual_norm_abs_sqrt, alpha=0.5);
        sns.regplot(
            x=self.y_predict,
            y=residual_norm_abs_sqrt,
            scatter=False, ci=False,
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        abs_sq_norm_resid = np.flip(np.argsort(residual_norm_abs_sqrt), 0)
        abs_sq_norm_resid_top_3 = abs_sq_norm_resid[:3]
        for i in abs_sq_norm_resid_top_3:
            ax.annotate(
                i,
                xy=(self.y_predict[i], residual_norm_abs_sqrt[i]),
                color='C3')
        ax.set_title('Scale-Location', fontweight="bold")
        ax.set_xlabel('Fitted values')
        ax.set_ylabel(r'$\sqrt{|\mathrm{Standardized\ Residuals}|}$');
        return ax

    def leverage_plot(self, ax=None):
        """
        Residual vs Leverage plot

        Points falling outside Cook's distance curves are considered observation that can sway the fit
        aka are influential.
        Good to have none outside the curves.
        """
        if ax is None:
            fig, ax = plt.subplots()

        ax.scatter(
            self.leverage,
            self.residual_norm,
            alpha=0.5);

        sns.regplot(
            x=self.leverage,
            y=self.residual_norm,
            scatter=False,
            ci=False,
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        leverage_top_3 = np.flip(np.argsort(self.cooks_distance), 0)[:3]
        for i in leverage_top_3:
            ax.annotate(
                i,
                xy=(self.leverage[i], self.residual_norm[i]),
                color = 'C3')

        xtemp, ytemp = self.__cooks_dist_line(0.5) # 0.5 line
        ax.plot(xtemp, ytemp, label="Cook's distance", lw=1, ls='--', color='red')
        xtemp, ytemp = self.__cooks_dist_line(1) # 1 line
        ax.plot(xtemp, ytemp, lw=1, ls='--', color='red')

        ax.set_xlim(0, max(self.leverage)+0.01)
        ax.set_title('Residuals vs Leverage', fontweight="bold")
        ax.set_xlabel('Leverage')
        ax.set_ylabel('Standardized Residuals')
        ax.legend(loc='upper right')
        return ax

    def vif_table(self):
        """
        VIF table

        VIF, the variance inflation factor, is a measure of multicollinearity.
        VIF > 5 for a variable indicates that it is highly collinear with the
        other input variables.
        """
        vif_df = pd.DataFrame()
        vif_df["Features"] = self.xvar_names
        vif_df["VIF Factor"] = [variance_inflation_factor(self.xvar, i) for i in range(self.xvar.shape[1])]

        print(vif_df
                .sort_values("VIF Factor")
                .round(2))


    def __cooks_dist_line(self, factor):
        """
        Helper function for plotting Cook's distance curves
        """
        p = self.nparams
        formula = lambda x: np.sqrt((factor * p * (1 - x)) / x)
        x = np.linspace(0.001, max(self.leverage), 50)
        y = formula(x)
        return x, y

In [3]:
def col_names(df):
    """
    Extract column names from dataframe, manipulate the column names into a 
    shortened form and then replace the names in the original dataframe
    
    Inputs
    ------
        df: (Pandas dataframe) dataframe with original column names
        
    Returns
    -------
        df: (Pandas dataframe) A dataframe with shorten new column names
        
    """

    replace_dict1 = {'\(Daily\)':"", "\(Hourly\)": "", "\[C]":"", "\[]":"", 
                     "Temperature ":"temp", "Air":"Air", "Drybulb":"", ":Zone":"",
                     "Operative":"Operative", "People Occupant Count ": "no_people", 
                     ' +':'_'}

    names2 = pd.DataFrame(df.columns, columns=['test'])

    names2['test']=names2['test'].replace(replace_dict1, regex=True)

    df.columns=names2['test']
    
    return(df)

In [4]:

# Extract out the appropriate parts of the date and time from a string column

def convert_24_to_00(df):
    """
    Midnight times in data set are recorded as 24:00 - Change the time to 00 
    and add 1 to the date.
    
    Inputs
    ------
        date_column: (Pandas series) containing string dates
        year: (int) Specifed year
        
    Returns
    -------
        A series containing python date times.
        
    """

    date_names = ['Month', 'Day', 'Time']

    df_date = df["Date/Time"].str.strip().str.split('\s+|/', expand=True)

    df_date.columns = date_names 
    df_date['Year'] = df["Year"]

    df_date["DMY"] = pd.to_datetime(df_date[['Day','Month','Year']])

    # Index those times with a value of 24:00:00

    cond = df_date['Time'] == '24:00:00'

    # Reset the 24:00:00 to 00:00:00

    df_date.loc[cond,'Time'] = '00:00:00'

    # If cond was try add one to the day

    df_date.loc[cond, 'DMY'] = df_date.loc[cond,'DMY'] + timedelta(days=1) 

    # Combine time and date

    return df_date['DMY']+ pd.to_timedelta(df_date['Time'])
    

In [5]:
def read_data(filepath, target = "Air", feature="Outdoor"):
    
    """
    Read schools temperature data in from a csv file selecting variables,
    create a variable that contains the year from the file name. Then select 
    variables that have "Date", "Year", "Outdoor", "Air Temperature" or 
    "People" in the variable names
    
    Inputs
    ------
        filepath: (string) Name  and path of the file to be read in
        target: (string) Which variable to choose as the predictand
        
    Returns
    -------
        A pandas dataframe containing the required variables
        
    """
    
    temperature_df = pd.DataFrame()
    
    for i_file in filepath:
        
        print("Read in data", i_file)
        
        df1 =  pd.read_csv(i_file,
                       parse_dates=True)
        
          
        
        # Extract year from title
        df1["Year"] = i_file.split("_TRY_")[1][0:4]
        
        #Subset to just the temperature and date columns
        
        #print("these are the column names\n", df1.columns)

        temperature_names = [i  for i in df1.columns if 
                             any(xs in i for xs in ["Date","Year",feature, 
                                                    target,"People"])]
        
        temperature_df = pd.concat([temperature_df,
                                    df1[temperature_names]])
        
        
    return temperature_df


In [6]:
def create_date_index(df):
    
    """
    Calls function to convert midnight times from 24:00 to 00:00 and sets 
    the result to be the index of the Pandas data frame
    
    Inputs
    ------
        df: (Pandas dataframe) Dataframe containing the schools data
        declared_year: set the year
        
    Returns
    -------
        df: (Pandas dataframe) A dataframe with the date variable set to
             be the index
        
    """

    df['datetime'] = convert_24_to_00(df)

    df.set_index('datetime', inplace=True)

    return df

In [7]:
def set_empty_classroom_temps_nan(df, target="Air"):
    
    """
    Set the variables containing the air temperature to be NAN if there are
    no people in the room. Drop the variable with the classroom occupancy
    from the file, as no longer needed after this step
    
    Inputs
    ------
        df: (Pandas dataframe) Dataframe containing the schools data
        target: (string)  Name of the variable that will be modelled
       
        
    Returns
    -------
        df: (Pandas dataframe) A dataframe with missing data if the room is 
             not occupied
        
    """
    #print("testing \n")
    #print(df.head())
    
    for i_com in G_COMPASS:
        
        index_empty= df[i_com + "_no_people"] <= 0

        df.loc[index_empty, i_com + '_' + target + "_temp"] = np.nan

        df.drop([i_com + "_no_people"], axis=1, inplace=True)

    return df

In [8]:
def daily_mean(df):
    """
    Calculates the daily mean and returns a dataset that just contains
    the daily means
    
    Inputs
    ------
        df: (Pandas dataframe) Dataframe containing the schools data
        
    Returns
    -------
        daily_x: (Pandas dataframe) A dataframe with the daily means
        
    """
    # Calculate the daily mean of the classroom temperatures
  
    daily_x = df.resample('D', closed='left').mean()

    return daily_x

In [9]:
def hourly_ts_plots(df1, labels=['Daily','Hourly'], target="Air"):
    
    """
    Produces time series plots of temperature data
    
    Inputs
    ------
        df: (Pandas dataframe) Dataframe containing the schools data
        declared_year: set the year
        target: (string)  Name of the variable that will be modelled
        
    Returns
    -------
        None
        
    """
    df=df1.copy()
    
    df = df.reset_index()
    
    df = df.dropna(axis=0)
    
    df['year'] =df['datetime'].dt.year
    df["day_of_year"] = df['datetime'].dt.dayofyear
    
    fig, ax =plt.subplots(2,2, sharey=True,figsize = (15, 15))

    plt.suptitle("Daily mean outdoor temperature and maximum operative temperature when classroom occupied") 
   
    fig.subplots_adjust(top=7)
    
    for count, i_com in enumerate(G_COMPASS):
        plt.subplot(2,2,count+1)
      
        #print("hourly_time series function:", df.columns)
        
        sns.lineplot(data=df, y=i_com + '_Outdoor_Air_temp', x='day_of_year',
                     hue = 'year', palette="tab10",style="year")
        sns.lineplot(data=df, y=i_com +'_' + target + '_temp', x='day_of_year',
                     hue = 'year', palette="tab10", style="year")
        
        plt.axhline(y=threshold, c="red")
        
        plt.title(i_com.title(), fontsize=G_LARGEFONT)
        plt.xlabel('Year', fontsize=G_MEDIUMFONT)
        plt.ylabel("Temperature",fontsize=G_MEDIUMFONT)
        plt.legend(loc="upper right", fontsize=G_MEDIUMFONT)
        plt.yticks(fontsize=G_SMALLFONT)

        fig.tight_layout()
       

In [10]:
def scatter_plots(df1, plot_title="", target="Air"):
    """
    Produces scatter plots of outdoor temperature vs Operative temperature
    
    Inputs
    ------
        df: (Pandas dataframe) Dataframe containing the schools data
        target: (string)  Name of the variable that will be modelled
        plot_title: (string)  title to use for the scatter plots
        
    Returns
    -------
        No returns
        
    """

    fig, ax = plt.subplots(2,2, sharey=True,figsize = (15, 15))
    
    df = df1.copy()
    
    df = df.reset_index()
    
    df = df.dropna(axis=0)
    
    df['year'] = df['datetime'].dt.year
    plt.suptitle(plot_title) 
    
    fig.subplots_adjust(top=7)
    for count, i_com in enumerate(G_COMPASS):
        plt.subplot(2,2,count+1)

        sns.scatterplot(data=df, y=i_com +'_' + target + '_temp', x=i_com +
                        '_Outdoor_Air_temp', hue = 'year', palette="tab10", 
                        style="year")
        
        plt.title(i_com.title(), fontsize=G_LARGEFONT)
        plt.xlabel('Outdoor Temperature', fontsize=G_MEDIUMFONT)
        plt.ylabel("Operative Temperature", fontsize=G_MEDIUMFONT)

        plt.yticks(fontsize=G_SMALLFONT)
        plt.xticks(fontsize=G_SMALLFONT)
        
        plt.tight_layout()

In [11]:
def remove_temps_le_12(df1):
    
    """
    Removes those observations from the dataframe where the outdoor temperature
    has a value less than of equal to 12
    
    Inputs
    ------
        df: (Pandas dataframe) Dataframe containing the schools data
        
    Returns
    -------
        df: (Pandas dataframe) A dataframe that only contains observations
            for when the outdoor temprature is greater than 12.
        
    """
    df=df1.copy()
    
    #print(df.head())
    for i_com in G_COMPASS:
        
        index_12 = df[i_com + "_Outdoor_Air_temp"] <= 12.0

        df.loc[index_12, i_com + "_Outdoor_Air_temp"] = np.nan

    df.dropna(inplace=True)

    return df


In [12]:
def one_slope_one_intercept(df, target="Air"):
    """
    Fit a regression model of the form a + bx (one intercept and slope)
    
    Inputs
    ------
        df: (Pandas dataframe) Dataframe containing the schools data
        target: (string)  Name of the variable that will be modelled
        
    Returns
    -------
        model1: (Object) Containing statistical information about the
        fitted model        
    """
    
    #print(df.columns)
    model1= smf.ols(formula = target + '_max_temp ~ outdoor_temp', data=df).fit()
    
    if L_OUTPUT == True:
        print("\n","="* 80, f'\n {Fore.RED}Model 1: Single line {Fore.BLACK} \n ******* \n')
        print(model1.summary())
        cls = Linear_Reg_Diagnostic(model1)
        fig, ax = cls()
    return model1


In [13]:
def one_slope_many_intercepts(df, target="Air"):
    """
    Fit a regression model of the form a + a_i + bx (many intercepts and one 
    slope). Where the different intercepts correspond to compass directions
    
    Inputs
    ------
        df: (Pandas dataframe) Dataframe containing the schools data
        target: (string)  Name of the variable that will be modelled
        
    Returns
    -------
        model2: (Object) Containing statistical information about the
        fitted model        
    """
    
    model2 = smf.ols(formula = target + '_max_temp ~ direction + outdoor_temp', data=df).fit()

    if L_OUTPUT == True:
        print("\n","="* 80, f'\n {Fore.RED}Model 2: Parallel lines {Fore.BLACK} \n ******* \n')
        print(model2.summary())
        cls = Linear_Reg_Diagnostic(model2)
        fig, ax = cls()
    return model2

In [14]:
def many_slopes_many_intercepts(df, target="Air"):
    """
    Fit a regression model of the form a + a_i + bx + b_i.x_i (many intercepts 
    and many slopes). Here each compass direction has a different intercept and  
    slope
    
    Inputs
    ------
        df: (Pandas dataframe) Dataframe containing the schools data
        target: (string)  Name of the variable that will be modelled
        
    Returns
    -------
        model3: (Object) Containing statistical information about the
        fitted model        
    """
    model3= smf.ols(formula = target + '_max_temp ~ direction + outdoor_temp*direction', data=df).fit()
    
    if L_OUTPUT==True:
        print("\n Model 3: Separate lines \n ******* \n")
        print("\n","="* 80, f'\n {Fore.RED}Model 3: Separate lines {Fore.BLACK} \n ******* \n')
        print(model3.summary())
        cls = Linear_Reg_Diagnostic(model3)
        fig, ax = cls()
    return model3




In [15]:
# Find best model out of max_mdl1, max_mdl2, max_mdl3
def find_best_model(candidate_models):
    """
    Based solely on the adjusted r-square statistic, find the "best model" and
    return the appropriate model object. 
    
    
    
    Inputs
    ------
        Candidate_models: (list) A list containing a set of candidate model
        objects
        
    Returns
    -------
         The  model object with the maximum adjusted r2 value     
    """    
    
    # The max function on a list of tuples returns the tuple which has the 
    # maximum value for the first element in the tuple compared to the other 
    # tuples.

    val, idx = max((val.rsquared_adj, idx) for (idx, val) in enumerate(candidate_models))
    
    return candidate_models[idx]

 
def find_threshold_outdoor_temp(best_model, threshold):#35):  
    """
    Calculate the outdoor temperature associated with an indoor temperature of 
    given threshold (e.g. 35oC) for the chosen model
     
    Inputs
    ------
        best model: (object) Containing statistical information associated with
        the best model
        threshold: (int) indoor temperature at which the school closes
        
    Returns
    -------
         Associated outdoor temperature for an indoor temperature equal to the
         threshold
    """    
    south_dummy = 0.0 
    south_interaction = 0.0

    south_parameters =best_model.params.filter(like='SOUTH', axis=0)
    no_south_params = len(south_parameters)
    
    if no_south_params == 1:
        south_dummy = south_parameters[0] 
    elif no_south_params == 2:
        south_dummy, south_interaction  = south_parameters
    elif no_south_params > 2:
        raise Exception("Too many parameters")


    outdoor_th_num = threshold - best_model.params.filter(like="Intercept",axis=0) - south_dummy
    outdoor_th_den = best_model.params.filter(like='outdoor_temp',axis=0) + south_interaction


    return outdoor_th_num[0]/outdoor_th_den[0]




In [16]:
def max_in_schoolday(df2, daily_data):

    """
    Find the maximum temperature in a school day
    
    Inputs
    ------
        daily_data: (Pandas dataframe) dataframe containing daily data
        df2: (Pandas dataframe) dataframe containing hourly data
        
    Returns
    -------
         daily_data: (Pandas dataframe)  dataframe containing dailydata  
    """   

    # Select columns and times between a start time and an endtime, find the maximum temperature

    day_temp_df = df2.loc[:,df2.columns.str.endswith('_temp')].between_time('08:00:00', '17:00:00')

    max_day_temp = day_temp_df.resample('D', closed='left').max()

    max_day_temp = max_day_temp.add_suffix('_max')

    daily_data = daily_data.join(max_day_temp)
    
    daily_data.dropna(how='all', axis=1, inplace=True)
    
    return daily_data


In [17]:
def mean_in_schoolday(df2, daily_data):

    """
    Find the mean temperature in a school day
    
    Inputs
    ------
        daily_data: (Pandas dataframe) dataframe containing daily data
        df2: (Pandas dataframe) dataframe containing hourly data
        
    Returns
    -------
         daily_data: (Pandas dataframe)  dataframe containing dailydata  
    """   

    # Select columns and times between a start time and an endtime, find the maximum temperature

    day_temp_df = df2.loc[:,df2.columns.str.endswith('_temp')].between_time('08:00:00', '17:00:00')

    mean_day_temp = day_temp_df.resample('D', closed='left').mean()

    mean_day_temp = mean_day_temp.add_suffix('_max')

    daily_data = daily_data.join(mean_day_temp)
    
    daily_data.dropna(how='all', axis=1, inplace=True)
    
    return daily_data


In [18]:
def data_2_long_thin(daily_data, df2, target = "Air"):
    """
    Convert data that is in short fat form to long thin form, so that we can
    fit a single statistical model
     
    Inputs
    ------
        daily_data: (Pandas dataframe) Containing statistical information associated with
        the best model
        df2: (Pandas dataframe) indoor temperature at which the school closes
        target: (string)  Name of the variable that will be modelled
        
    Returns
    -------
         df_long_thin: (Pandas dataframe) Required data (temperature, maximum temperature and 
         direction) in long thin format    
    """   
    
    df_long_thin = daily_data.melt(
        value_vars=['WEST_Outdoor_Air_temp', 'NORTH_Outdoor_Air_temp', 
                    'EAST_Outdoor_Air_temp', 'SOUTH_Outdoor_Air_temp'], 
        value_name='outdoor_temp', var_name='direction')
    
    target_names = [i_com + '_' + target + '_temp' for i_com in G_COMPASS]
    
    target_names_max =  [i_com + '_' + target + '_temp_max' for i_com in G_COMPASS]
    
    #print(daily_data.columns)
    
    df_long_thin[target + '_temp'] = daily_data.melt(value_vars=target_names)['value']
    
    df_long_thin[target + '_max_temp'] = daily_data.melt(value_vars=target_names_max)['value']
    
    df_long_thin['direction'] = df_long_thin['direction'].str.split('_', n=1).str[0]
    
    return df_long_thin




In [19]:
def save_to_excel(filename, threshold, i_count):
    
    """
    Save data for climada impact functions file to excel spreadsheet
    
    For climada the spreadsheet needs to have 7 columns: intensity, mdd, paa, 
    peril_id, intensity_unit, name and impact_fun_id
    
    intensity: is  the boundary points of the ranges at which the mdd applies
    mdd: mean degree of damage in this example is set to 0 or 1.
         1 represents school closure, 0 school open.
    paa: proportion of assets affected is set to 1
    peril_id: Set to HS for Heat Stress
    intensity_unit: set to DegC, Degrees Celsisus
    name: Set to Heat stress
    impact_fun_id: Unique identifier for each impact function, this is based on
                   region, architype (5 categories that relate to when the 
                   school was built, and school type (secondary or primary)
    
     
    Inputs
    ------
        filename: (string) Name of file to store data into
        threshold:(float) Outside temperature threshold
        i_count: (int) Impact function number
        
      
    """  
    
    df = pd.DataFrame(data=[11.5, threshold - 0.00001, threshold , 50],
        columns=['intensity'])
    
    df['mdd'] =(df["intensity"] >= threshold)*1
    df['paa'] = 1
    df['peril_id'] = 'HS'
    df['intensity_unit'] = 'DegC'
    df['name'] = 'Heat Stress'
    df.insert(0, 'impact_fun_id', i_count + 1)
    
    df.reset_index()
    
    # First time through open and write to file. Subsequently append
    # to file
    if i_count == 0:    
        with pd.ExcelWriter(filename, engine="openpyxl")  as writer:
            df.to_excel(writer, sheet_name="impact_functions", index=False)
    else:
        with pd.ExcelWriter(filename, engine="openpyxl",mode="a", if_sheet_exists="overlay")  as writer:
            df.to_excel(writer,sheet_name="impact_functions", index=False, header=False, 
                        startrow=writer.sheets['impact_functions'].max_row)
         

In [20]:
def check_filenames():
    """
    Create a list of existing valid filenames. 
    
    Organised such that files for the same
    school_type (secondary or primary), regions (one of 13), construction type 
    (era when it was built) are grouped together across simulation_type.
    
    The intention is to pool the simulation types together to create regression
    relations that are valid for all simulation types.
     
    Inputs
    ------
        
        
    Returns
    -------
    set_names: (Panda series) containing appropriate references to files    
    """  
    
    # Directory where the DoE temperature files are stored
    data_dir = '/data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/'
    
    set_names = pd.DataFrame()
    
    # Variables used in file names to describe the data
    school_type = ['P', 'S']
    region_type = ["Z"+str(i).zfill(2) for i in range(1,14)]
    construction_type = [str(i).zfill(2) for i in range(5)]
    construction_dict = {'_00_': 'Pre-1918', '_01_': "1919-1945",
                         '_02_':'1945-1967', '_03_': "1967-1976",
                         '_04_':'Post-1976'}
    simulation_type_dict = {'2020':'High', '2050':'Medium', '2080':'Medium'}
    ventilation_type = ['Nat']
    simulation_type= ['2020', '2050', '2080']
    building_type = ['Base']
    operation_type = ['BaseOp']
    end_str = ['/eplusout.csv']
    
    # Produce a list of all possible files.
    
    sub_dir2 = list(product(school_type, region_type, construction_type,
                        simulation_type, ventilation_type))
    
    
    file_names_all = [data_dir +  i_file[1] + "_TRY_" + i_file[3] + 
              simulation_type_dict[i_file[3]] + "50_/" + "_".join(i_file[0:3]) + 
               "_" + i_file[4] + "/eplusout.csv" for i_file in sub_dir2]

    
    # Remove filenames which DoE haven't supplied, 
    # Create an unique identifier that ignores simulation type, as we
    # are going to create impact equations that are valid across simulate type
    # This code groups files that are the same except for simulation type
    # together
    
    file_name_exist = [[i_count//3, i_file2] for i_count, i_file2 in 
                   enumerate(file_names_all) if os.path.exists(i_file2) is True]
    
    df_filename =  pd.DataFrame(file_name_exist)

    set_names["files"] = df_filename.groupby([0]).agg({1:list})
    
    return set_names

In [21]:
def paper_plots(df, lt, model, outdoor_threshold, out_plots, threshold,
                compass_dir="SOUTH", target="Air"):
    """
    Produces three plots for use in paper and saves the output to 
    "/data/users/ldawkins/UKCR/SchoolApp/plots" by default
    
    
    The following three plots are produced:
    1) Time series of maximum daily classroom temperature and mean daily
       (outdoor) temperature
    2) Scatter plot of Daily mean temperature(outdoor) and daily maximum
       class room temperature
    3) Step function which shows the daily mean temperature at which the 
       classroom threshold will be exceeded
    
    Inputs
    ------
        df: (Pandas dataframe) Required data in short fat format
        lt: (Pandas dataframe) Required data (temperature, maximum temperature 
             and direction) in long thin format 
        model: model object associate with best fitting linear regression model
        outdoor_threshold: Outside temperature threshold 
        out_plots: (string) name of file in which to store plots
        target: (string)  Name of the variable that will be modelled
        
    Returns
    -------
        No returns
        
    """
    
    outplots_path = "/data/users/ldawkins/UKCR/SchoolApp/plots" + out_plots 
    
    lt["fitted"]= model.fittedvalues

    df_lt_dir = lt[lt["direction"] == compass_dir]
    df_lt_dir["classified"]= (df_lt_dir["outdoor_temp"] >= outdoor_threshold) * 1.0
    
    df_dir = df.loc[:, df.columns.str.startswith(compass_dir)]

    df_dir = df_dir.reset_index()

    df_dir["day_of_year"] = df_dir['datetime'].dt.dayofyear
    
    fig, ax = plt.subplots(3,1,figsize = (4, 6))
    
    plt.subplot(3,1,1)
    #print(df_dir.columns)
    sns.scatterplot(data=df_dir, y='SOUTH_Outdoor_Air_temp', x='day_of_year')
    sns.scatterplot(data=df_dir, y='SOUTH_' + target + '_' + 'temp_max', x='day_of_year')
    
    # plt.title(compass_dir, fontsize=G_LARGEFONT)
    plt.xlabel('Day of Year', fontsize=G_SMALLFONT)
    plt.ylabel("Temperature", fontsize=G_SMALLFONT)

    plt.yticks(fontsize=G_SMALLFONT)
    plt.xticks(fontsize=G_SMALLFONT)
    
    #print(df_lt_dir.columns)
    plt.subplot(3,1,2)

    sns.scatterplot(data=df_lt_dir, x='outdoor_temp', y= target + '_max_temp')
    sns.lineplot(data=df_lt_dir, x='outdoor_temp', y='fitted')

    plt.axhline(y=threshold, c="red")
    
    plt.xlabel('Daily mean temperature', fontsize=G_SMALLFONT)
    plt.ylabel("Daily max operative temp", fontsize=G_SMALLFONT)

    plt.yticks(fontsize=G_SMALLFONT)
    plt.xticks(fontsize=G_SMALLFONT)
    
    plt.subplot(3,1,3)

    sns.lineplot(x=[min(df_lt_dir['outdoor_temp']), outdoor_threshold * 0.999999, 
                outdoor_threshold, max(df_lt_dir['outdoor_temp'])], y=[0,0,1,1], 
                 color="black")
    
    plt.xlabel('Daily mean temperature', fontsize=G_SMALLFONT)
    plt.ylabel("Threshold exceeded", fontsize=G_SMALLFONT)

    plt.yticks(fontsize=G_SMALLFONT)
    plt.xticks(fontsize=G_SMALLFONT)
    
    plt.tight_layout()
    plt.savefig(outplots_path + "_" + compass_dir + ".png")
    
    #sns.scatterplot(data=df_dir, x='outdoor_temp', y='classified')
  

In [22]:
def names_for_model_params(path_to_file, index_type):
    construction_dict = {'00': 'Pre-1918', '01': "1919-1945",
                         '02':'1945-1967', '03': "1967-1976",
                         '04':'Post-1976'}
    school_dict = {'P':'Primary', 'S':'Secondary'}
    region_dict = {'Z01':'Thames Valley', 'Z02':'South East', 'Z03':'Southern',
                   'Z04':'South West', 'Z05':'Severn Valley', 'Z06':'Midlands',
                   'Z07':'West Pennines', 'Z08':'North West', 'Z09':'Borders',
                   'Z10':'North East', 'Z11':'East Pennines', 
                   'Z12':'East Anglia', 'Z13':'Wales'}
    name_col = [ i_name for i_name in path_to_file.split("/")
                           if "Nat" in i_name][0].split("_")[0:3]
    col_parameters = pd.Series([index_type, school_dict[name_col[0]], 
                                region_dict[name_col[1]],
                                construction_dict[name_col[2]]],
                               index=['Architype Number', 'Primary/Secondary', 'Region', 
                                      'Building age'])
    return(col_parameters)





In [23]:
def parameter_output(model, threshold, meta_school, filename):
    output_params = pd.concat([meta_school, model.params])
    output_params.loc['outdoor threshold'] = threshold
    
    r2_stat = pd.Series([max_mdl1.rsquared_adj, max_mdl2.rsquared_adj,
                         max_mdl3.rsquared_adj], index= ["Single line - adj R2",
                                                         "Parallel lines - adj R2",
                                                         "Multiple lines - adj R2"])
    
    output_params = pd.concat([output_params, r2_stat])
    output_row = pd.DataFrame(output_params).T
    
    # First time through open and write to file. Subsequently append
    # to file
    if i_count == 0:    
        with pd.ExcelWriter(filename, engine="openpyxl")  as writer:
            output_row.to_excel(writer, sheet_name="impact_functions", index=False)
    else:
        with pd.ExcelWriter(filename, engine="openpyxl",mode="a", if_sheet_exists="overlay")  as writer:
            output_row.to_excel(writer,sheet_name="impact_functions", index=False, header=False, 
                        startrow=writer.sheets['impact_functions'].max_row)
    
    return 

In [24]:
#Main program
"""
This program reads in temperature data as supplied by the Department for
education and creates impact functions that relate outdoor temperature to 
indoor temperature.  It then finds the threshold outdoor temperature which
corresponds to an indoor temperature of X degrees as this is the temperature
at which the school closes.

These relationships are saved to a file in a format that is suitable for 
climada to read in.

"""

pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

# Set global variables
G_COMPASS = ["NORTH", "EAST", "SOUTH", "WEST"]
G_SLARGEFONT=25
G_LARGEFONT=14
G_MEDIUMFONT=12
G_SMALLFONT= 8
L_OUTPUT = False
L_SAVE = True

file_path = check_filenames()

no_files = 'many'  # 'one' or 'many'
predictand = "Operative" ; #"Operative or Air"
predictor = "Outdoor"
threshold = 26 #26 or 35
meanmax = 'max' # 'mean' or 'max' - which indoor metric to model

file_output = predictand + str(threshold) + "_temperature_relationships_" + meanmax + ".xlsx"
L_PAPER_GRAPHS = False

print("\n ******* \n Target variable is:", predictand, 
      "temperature \n ")

### Options to specify if it is desirable to only analyse a single climate 
### region.

if no_files =='one':
    
    # Specify Primary or Secondary P or S
    school_type = 'P'   
    
    # Specify architype: 00': 'Pre-1918', '01': "1919-1945", '02':'1945-1967',
    #                   '03': "1967-1976", '04':'Post-1976' 
    construction_type = '00' 
    
    ## Specify region:
    #  1:'Thames Valley', 2:'South East',    3:'Southern',
    #  4:'South West',    5:'Severn Valley', 6:'Midlands',
    #  7:'West Pennines', 8:'North West',    9:'Borders',
    # 10:'North East',   11:'East Pennines', 
    # 12:'East Anglia',  13:'Wales'
    i_region = 2  #
    
    L_OUTPUT = True
    L_PAPER_GRAPHS = True
    
    ######  No user specified options below here ######
      
    ventilation_type = ['Nat']
    region_type = "Z" + str(i_region).zfill(2)
    
    string = school_type + '_' + region_type + '_' + construction_type
    
    file_path = file_path[file_path["files"].str[0].str.contains(string) == True]
    
# Loop round different files creating separate relations for each region, 
# school type and architype

for i_count in file_path.index:
    
    list_files = file_path.loc[i_count][0]
    
    if L_OUTPUT == True:
        print(names_for_model_params(list_files[0], i_count),"\n ****** \n")
       
    # Read in the data
    df_schools = read_data(list_files, target = predictand, feature=predictor)
    
    if df_schools.empty:
        print("No files found for: \n", i_count)
        continue
        
    # Make shorter variable names for data
    df_schools = col_names(df_schools) 
    
    # create a date index and sort out times recorded as 24:00
    df_schools = create_date_index(df_schools)
    
    # Remove observations when there are no pupils in the class
    df_schools = set_empty_classroom_temps_nan(df_schools, target=predictand)
  
    # Calculate the daily mean temperature
    df_daily = daily_mean(df_schools)
       
    # Find maximum temperature in a day
    if meanmax == 'max':
        df_daily = max_in_schoolday(df_schools, df_daily)
    else:
        df_daily = mean_in_schoolday(df_schools, df_daily)
    
    # Remove observations with an outdoor temperature less than 12
    df_daily_gt_12 = remove_temps_le_12(df_daily)
    
    # Organise data from short fat to long thin
    long_thin = data_2_long_thin(df_daily_gt_12, df_schools, target=predictand)
    
    # Fit regression models
    max_mdl1 = one_slope_one_intercept(long_thin, target=predictand)
    max_mdl2 = one_slope_many_intercepts(long_thin, target=predictand)
    max_mdl3 = many_slopes_many_intercepts(long_thin, target=predictand)
    
    # Find the best model as defined by the adjusted r-square statistic
    b_model = find_best_model([max_mdl1, max_mdl2, max_mdl3])
    
    if L_OUTPUT == True:
        print("\n","="* 80, f'\n {Fore.RED}Best model based on adjusted R-rsquared statistic is {Fore.BLACK} \n ******* \n')
        print(b_model.summary())
  
    # Find the outdoor temperature associated with the indoor threshold
    outdoor_th = find_threshold_outdoor_temp(b_model, threshold=threshold)
    
    # Save relationships to an excel file
    if L_SAVE == True:
        save_to_excel(file_output, outdoor_th, i_count)
        
    # Save parameters to an excel file
    region_architype_name = names_for_model_params(list_files[0], i_count)
        
    parameter_output(b_model, outdoor_th, region_architype_name, filename=file_output.replace("s_","_parameters_"))

    if L_OUTPUT == True:
        print("\n","="* 80, f'\n {Fore.RED}Plots for all observations {Fore.BLACK} \n ******* \n')
        print(df_daily.head())
        hourly_ts_plots(df_daily, labels=['External','Internal'], target="Operative", threshold=threshold)
        plot_title = "Operative temperature and outdoor temperture when classroom occupied"
        scatter_plots(df_daily, plot_title=plot_title, target=predictand)
        
        print("\n","="* 80, f'\n {Fore.RED}Plots restricted to obsservations where the outdoor air temperature > 12 {Fore.BLACK} \n ******* \n')
        plot_title="Daily mean outdoor temperature and maximum operative temperature when classroom occupied when outdoor air temp > 12" 
        scatter_plots(df_daily_gt_12, plot_title=plot_title, target=predictand)
        
    if L_PAPER_GRAPHS == True:
        paper_plots(df_daily, long_thin, b_model, outdoor_th, string, target=predictand, threshold=threshold)
        
print("End of program")
    


 ******* 
 Target variable is: Operative temperature 
 
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z01_TRY_2020High50_/P_Z01_00_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z01_TRY_2050Medium50_/P_Z01_00_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z01_TRY_2080Medium50_/P_Z01_00_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z01_TRY_2020High50_/P_Z01_01_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z01_TRY_2050Medium50_/P_Z01_01_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z01_TRY_2080Medium50_/P_Z01_01_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/Da

Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z05_TRY_2020High50_/P_Z05_00_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z05_TRY_2050Medium50_/P_Z05_00_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z05_TRY_2080Medium50_/P_Z05_00_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z05_TRY_2020High50_/P_Z05_01_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z05_TRY_2050Medium50_/P_Z05_01_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z05_TRY_2080Medium50_/P_Z05_01_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOf

Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z08_TRY_2020High50_/P_Z08_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z08_TRY_2050Medium50_/P_Z08_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z08_TRY_2080Medium50_/P_Z08_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z09_TRY_2020High50_/P_Z09_00_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z09_TRY_2050Medium50_/P_Z09_00_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z09_TRY_2080Medium50_/P_Z09_00_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOf

Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z12_TRY_2080Medium50_/P_Z12_02_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z12_TRY_2020High50_/P_Z12_03_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z12_TRY_2050Medium50_/P_Z12_03_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z12_TRY_2080Medium50_/P_Z12_03_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z12_TRY_2020High50_/P_Z12_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z12_TRY_2050Medium50_/P_Z12_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOf

Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z03_TRY_2020High50_/S_Z03_02_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z03_TRY_2080Medium50_/S_Z03_02_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z03_TRY_2020High50_/S_Z03_03_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z03_TRY_2080Medium50_/S_Z03_03_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z03_TRY_2020High50_/S_Z03_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z03_TRY_2080Medium50_/S_Z03_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffi

Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z07_TRY_2080Medium50_/S_Z07_02_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z07_TRY_2020High50_/S_Z07_03_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z07_TRY_2050Medium50_/S_Z07_03_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z07_TRY_2080Medium50_/S_Z07_03_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z07_TRY_2020High50_/S_Z07_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z07_TRY_2050Medium50_/S_Z07_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOf

Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z11_TRY_2020High50_/S_Z11_03_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z11_TRY_2050Medium50_/S_Z11_03_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z11_TRY_2080Medium50_/S_Z11_03_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z11_TRY_2020High50_/S_Z11_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z11_TRY_2050Medium50_/S_Z11_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOffice/Z11_TRY_2080Medium50_/S_Z11_04_Nat/eplusout.csv
Read in data /data/users/ldawkins/UKCR/DataForSchoolApp/UCL_model_data/UCL_archetypes/118091-MetOf

In [132]:
# !git status

In [133]:
jupyter_file_name = "_".join(region_architype_name[1:4]).replace(" ","_")
!jupyter nbconvert --output=$jupyter_file_name --output-dir="~hadkb/SIRA/Heat_stress/Python/school_temp_output" --to pdf --TemplateExporter.exclude_input=True school_temp_analysis_to_excel.ipynb
         

[NbConvertApp] Converting notebook school_temp_analysis_to_excel.ipynb to pdf
[NbConvertApp] Writing 54160 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 21976 bytes to /home/h05/hadkb/SIRA/Heat_stress/Python/school_temp_output/Secondary_Wales_1945-1967.pdf
